In [3]:
import pandas as pd

In [4]:
# Import resale flat price data
df_raw = pd.read_csv('../data/resale_flat_prices.csv')
df_raw

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
164315,2023-05,YISHUN,EXECUTIVE,827,YISHUN ST 81,01 TO 03,154.0,Maisonette,1987,63 years 06 months,829000.0
164316,2023-05,YISHUN,EXECUTIVE,824,YISHUN ST 81,07 TO 09,146.0,Maisonette,1987,63 years 06 months,855000.0
164317,2023-07,YISHUN,EXECUTIVE,837,YISHUN ST 81,01 TO 03,146.0,Maisonette,1988,63 years 07 months,860000.0
164318,2023-01,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,07 TO 09,179.0,Multi Generation,1987,63 years 11 months,1080000.0


In [15]:
import requests
import json

def find_postal(lst):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    dfs = []
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        data = json.loads(response.text) 
    
        temp = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp["address"] = add
        
        # Append to the list of DataFrames
        dfs.append(temp)

    # Concatenate all DataFrames in the list into one DataFrame
    result_df = pd.concat(dfs, ignore_index=True)

    return result_df

In [ ]:
df_raw['address'] = df_raw['block'] + " " + df_raw['street_name']
df_raw

In [21]:
unique_addresses = list(set(df_raw['address'].unique()))
len(unique_addresses)

9493

In [28]:
a = unique_addresses[:5]
a

['915 JURONG WEST ST 91',
 '832 TAMPINES ST 82',
 '112 YISHUN RING RD',
 '362 YUNG AN RD',
 '872 WOODLANDS ST 81']

In [29]:
b = find_postal(a)
b

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=915 JURONG WEST ST 91
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=832 TAMPINES ST 82
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=112 YISHUN RING RD
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=362 YUNG AN RD
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=872 WOODLANDS ST 81


,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,NANYANG EMERALD,915,JURONG WEST STREET 91,NANYANG EMERALD,915 JURONG WEST STREET 91 NANYANG EMERALD SING...,640915,11813.1631195759,35916.8260860052,1.34108918953192,103.687870332112,103.687870332112,915 JURONG WEST ST 91
1,TAMPINES PALMWALK,832,TAMPINES STREET 82,TAMPINES PALMWALK,832 TAMPINES STREET 82 TAMPINES PALMWALK SINGA...,520832,39360.2836844912,36913.1424747147,1.350101714776,103.935397838075,103.935397838075,832 TAMPINES ST 82
2,CHONG PANG VALE,112,YISHUN RING ROAD,CHONG PANG VALE,112 YISHUN RING ROAD CHONG PANG VALE SINGAPORE...,760112,27525.5259930072,46208.5996352285,1.43416864539746,103.829054979214,103.829054979214,112 YISHUN RING RD
3,HDB PUBLIC SHELTERS,362,YUNG AN ROAD,HDB PUBLIC SHELTERS,362 YUNG AN ROAD HDB PUBLIC SHELTERS SINGAPORE...,610362,15373.1379570095,35381.3230111972,1.33624800193943,103.719858973785,103.719858973785,362 YUNG AN RD
4,362 YUNG AN ROAD SINGAPORE 610362,362,YUNG AN ROAD,NIL,362 YUNG AN ROAD SINGAPORE 610362,610362,15364.7265448701,35371.7642911337,1.33616155292127,103.719783396486,103.719783396486,362 YUNG AN RD
5,872 WOODLANDS STREET 81 SINGAPORE 730872,872,WOODLANDS STREET 81,NIL,872 WOODLANDS STREET 81 SINGAPORE 730872,730872,23158.3441046439,47305.4007053953,1.44408730145683,103.789811521995,103.789811521995,872 WOODLANDS ST 81


In [26]:
lst = ['18 CHOA CHU KANG GROVE']

In [27]:
c = find_postal(lst)
c

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=18 CHOA CHU KANG GROVE


,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,LONGTITUDE,address
0,SOL ACRES,18,CHOA CHU KANG GROVE,SOL ACRES,18 CHOA CHU KANG GROVE SOL ACRES SINGAPORE 688211,688211,18747.0412725797,39792.7647733055,1.37614468896726,103.750174045192,103.750174045192,18 CHOA CHU KANG GROVE


In [ ]:
origin_destination = [[(1.3772, 103.9343), (1.3761, 103.7502)],
                     ]